# USU Australs Trial Debate Allocation Algorithm
Developed by Viran Weerasekera (2017-18)

In [ ]:
import gurobi as grb
import pandas as pd
import numpy as np
import math
import time

def allocate():
    global times
    alltimes = {1:"9:00am",2:"10:15am",3:"11:30am",4:"1:15pm",5:"2:30pm",6:"3:45pm",7:"5:00pm",8:"6:15pm"}
    
    # Import data
    global availabilities
    availabilities = {}
    global vetoes
    vetoes = {}
    global speakers
    speakers = []
    
    csv = pd.read_csv("Australs 2019 Trial Registration (Responses) - Form responses 1.csv",index_col=0)

    for index,row in csv.iterrows():
        if row[6] != "Adjudicating only":
            speakers.append(row[0])
            vetoes[row[0]] = row[9]
            availabilities[row[0]] = {}
            availabilities[row[0]][1] = row[10]
            availabilities[row[0]][2] = row[11]
            availabilities[row[0]][3] = row[12]
            availabilities[row[0]][4] = row[13]
            availabilities[row[0]][5] = row[14]
            availabilities[row[0]][6] = row[15]
            availabilities[row[0]][7] = row[16]
            availabilities[row[0]][8] = row[17]
        
    #Calculate how many timeslots are required
    alltimeslots = (1,2,3,4,5,6,7,8)
    debatesreqd = int(np.floor(len(speakers)/6))
    timeslots = alltimeslots[:debatesreqd]
    times = {}
        
    #Form Feasible Team Set (FTS)
    fts = {}
    index = 0
    for speaker1 in speakers:
        for speaker2 in speakers:
            for speaker3 in speakers:
                if speaker1 != speaker2 and speaker1 != speaker3 and speaker2 != speaker3:
                    if vetoes[speaker1] != "1st Speaker" and vetoes[speaker2] != "2nd Speaker" and vetoes[speaker3] != "3rd Speaker":
                        fts[index] = [speaker1,speaker2,speaker3]
                        index += 1
    
    #Generate Costs
    costs = {}
    for key,time in alltimes.items():
        if key in timeslots:
            times[key] = time

    for key,team in fts.items():
        for slot in timeslots:
            costs[key,slot] = 0

        #Preference
        for speaker in team:
            for slot in timeslots:
                if availabilities[speaker][slot] == "Prefer":
                    costs[key,slot] -= 100

        # Vetoes - Separate from/actioned after Preference because it should override all "Prefer"
        for speaker in team:
            for slot in timeslots:
                if availabilities[speaker][slot] == "Unavailable":
                    costs[key,slot] = grb.GRB.INFINITY
                    
        for speaker in team:
            rlvconflicts = []
            for conflictobject in conflicts:
                if speaker in conflictobject:
                    rlvconflicts.append(conflictobject)
        for rlvconflict in rlvconflicts:
            for spk in rlvconflict:
                if spk in team:
                    for slot in timeslots:
                        costs[key,slot] = grb.GRB.INFINITY
                    
    #Optimisation
    a = grb.Model("2018 Australs Trial Allocator")
    x = {}
    for key,value in costs.items():
        x[key] = a.addVar(vtype = grb.GRB.INTEGER, obj = value, name = "x_{}".format(key))

    #Exactly 2 teams in each timeslot
    for slot in timeslots:
        a.addConstr(sum(x[team,slot] for team in fts.keys()) == 2)

    #Each team to no more than 1 timeslot
    for key,team in fts.items():
        a.addConstr(sum(x[key,slot] for slot in timeslots) <= 1)

    thirdvetoes = []
    for s in speakers:
        affectedTeams = []
        for key,team in fts.items():
            if s in team:
                affectedTeams.append(key)

        #Each speaker to no more than 1 debate in 1 timeslot
        a.addConstr(sum(x[team,slot] for team in affectedTeams for slot in timeslots) <= 1)

        if vetoes[s] == "3rd Speaker":
            thirdvetoes.append(s)

    #print(thirdvetoes)
    for speaker in thirdvetoes:
        affectedTeams2 = []
        for key,team in fts.items():
            if speaker in team:
                affectedTeams2.append(key)
        #3rd-vetoes must be in exactly 1 timeslot (given they can't be tacked onto the end as alternate 3rd)
        a.addConstr(sum(x[team,slot] for team in affectedTeams2 for slot in timeslots) == 1)

    a.modelSense = grb.GRB.MINIMIZE
    a.optimize()
  
    allocatedSpeakers = []
    unallocatedSpeakers = []
    print("-----")
    print("USU AUSTRALS TRIALS 2019 - DEBATE ALLOCATION")
    print("Speakers in each team are listed in speaking order.")
    print("Sides for each debate should be determined randomly.")
    print("-----")
    for key,value in costs.items():
        if x[key].x != 0:
            print("Team {} {} at {} (Cost {})".format(key[0],fts[key[0]],times[key[1]],value))
            for speaker in fts[key[0]]:
                allocatedSpeakers.append(speaker)

    for speaker in speakers:
        if speaker not in allocatedSpeakers:
            unallocatedSpeakers.append(speaker)
    print("-----")
    print("Successfully allocated {} out of {} speakers across {} debates".format(len(allocatedSpeakers),len(speakers),debatesreqd))
    print("Unallocated Speakers: {}".format(unallocatedSpeakers))
    print("-----")
    for unallocatedSpeaker in unallocatedSpeakers:
        report(unallocatedSpeaker)
    a.write("2018 Australs Allocation.lp")
    
    
def report(speaker):
    fname = speaker.split(" ")[0]
    if speaker not in speakers:
        print("Speaker '{}' does not exist.".format(fname))
    else:
        if vetoes[speaker] == "I would not like to veto any position":
            vetoString = "{} has no speaker position vetoes.".format(fname)
        else:
            vetoString = "{} has vetoed {}.".format(fname,vetoes[speaker])
            
        print("Report for {}".format(speaker))
        print(vetoString)
        print("{}'s availability is as follows:".format(fname))
        for slot,availability in availabilities[speaker].items():
            try:
                print("{}: {}".format(times[slot],availability))
            except:
                pass
        print("-----")
    
conflicts = []
def addConflict(conflicter,conflictee):
    conflicts.append([conflicter,conflictee])

In [ ]:
allocate()